In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import yaml
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime
from datetime import datetime
from datetime import date

import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

import os
from os import listdir


# Google Cloud Language Translation API
# We're using the basic version here == "v2" 
from google.cloud import translate_v2

import timeit

## nltk imports
# ! pip install gensim
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /Users/euniceliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/euniceliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset used in this notebook
- read in: H_2A_Disclosure_Data_FY2020.xlsx & and H_2A_Disclosure_Data_FY2021.xlsx (I need the H-2A Job Certificate Data in year 2020 and 2021 because first the years correspond to the addendum data as I include the recent 2020 and 2021 year data and moreover I am specifically interested in the "CASE_STATUS" column that enables me to understand that whether when a employer files an application to hire workers, their cases are approved or not. 
- export & read in: h2a_combined_20202021_int.csv (the concatenated H_2A_Disclosure_Data_FY2020.xlsx and H_2A_Disclosure_Data_FY2021 data )
- read in: translated_job_postings (the job addendum data that continues all the translated addendum. in other words non-English addendum are all translated and presented in the dataset)
- export: combined_certificate_postings.csv (the merged data that contains the job addendums and the CASE_STATUS variables)


# Major tasks accomplished in this notebook
- 1) clean the H-2A Job Certificate Data 
- 2) merge the H-2A Job Certificate Data (include the outcome variables "CASE_STATUS" that we want to predict) with the job addendums data

# Why are these tasks important
- Originally, the outcome variables (CASE_STATUS) and addendums are in separate data files, pre-merging cleaning are crucial for combining those two datasets together as the merged dataset would allow us to run the predictive model as each case has a outcome and corresponding text.

# Directory

In [2]:
# dropbox
dropbox_general = "/Users/euniceliu/Dropbox (Dartmouth College)/"
DROPBOX_DATA_PATH = os.path.join(dropbox_general,
                                "qss20_finalproj_rawdata/summerwork/")
DATA_RAW_DIR = os.path.join(DROPBOX_DATA_PATH, "raw/")
DATA_ID_DIR = os.path.join(DROPBOX_DATA_PATH, "intermediate/")
WRITEFOLDER = os.path.join(DATA_ID_DIR)
# github
GITHUB_DATA_PATH = "../data/raw_data/"

## Clean and Merge the the H-2A Job Certificate Data Across Different Years 

### Read in Dataset & Combined into a list
- We are only interested in 2020 and 2021 job certificate data
because the job addendumns data we have it's only from 2020-2021

In [2]:
df_2020 = pd.read_excel(DATA_RAW_DIR+"H_2A_Disclosure_Data_FY2020.xlsx")
df_2021 = pd.read_excel(DATA_RAW_DIR+"H_2A_Disclosure_Data_FY2021.xlsx")
full_data = [df_2020, df_2021]

### Find and print column names in 2021 that were not present in 2020


In [3]:
# check and print column names in 2021 that were not present in 2020
# these columns are not the main variables that we are focusing on (which is CASE_STATUS)
# so it is okay when merging the two years' data to disregard these columns
list(full_data[1].columns.difference(full_data[0].columns))

['ADDENDUM_C_ATTACHED',
 'H-2A_LABOR_CONTRACTOR',
 'HOURLY_SCHEDULE_BEGIN',
 'HOURLY_SCHEDULE_END',
 'MEALS_CHARGE',
 'TOTAL_WORKSITES_RECORDS']

### Merge 2020 and 2021 H-2A Job Certificate Data

In [4]:
# concatentate the 2020 and 2021 data on only the intersecting columns
intersect_2021 = list(set(df_2020.columns).intersection(df_2021.columns)) 
h2a_combined_20202021 = pd.concat([df_2020, df_2021]) 
h2a_combined_20202021_int = h2a_combined_20202021[intersect_2021].copy()

In [5]:
# export the data to csv file
h2a_combined_20202021_int.to_csv(DATA_ID_DIR+'h2a_combined_20202021_int.csv')

## Merge the Job Certificate Data with the Job Addendums Data

### Read in the data

In [6]:
# read in the merged H2-A Job Certificate Data
h2ajob_certificate=pd.read_csv(DATA_ID_DIR+"h2a_combined_20202021_int.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (46,69,105) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Deal with the CASE_STATUS column particulary in the Job Certificate Data before before (since that is our outcome variables)
- There are multiple duplicated case studies so in the follow code: 
1. I checked that if all the repeated case numbers have the same outcome (CASE_STATUS)
2. Dropped the duplicated (keep the first case number and status)
3. Deleted the "withdrawn" CASE_STATUS because it is hard to classify that into either certified or denied categories

In [7]:
# how many unique values in the CASE_STATUS column?
h2ajob_certificate['CASE_STATUS'].nunique()
# print the unique values of the CASE_STATUS column 
print(h2ajob_certificate['CASE_STATUS'].unique())

6

['Determination Issued - Certification' 'Determination Issued - Withdrawn'
 'Determination Issued - Denied'
 'Determination Issued - Partial Certification'
 'Determination Issued - Certification (Expired)'
 'Determination Issued - Partial Certification (Expired)']


### Investigate cases that have the same case number but different outcomes

In [9]:
diffoutcomes = h2ajob_certificate.groupby('CASE_NUMBER').CASE_STATUS.nunique() > 1
diffoutcomes[diffoutcomes].index.tolist()

['H-300-19297-107918',
 'H-300-19310-128739',
 'H-300-19346-194783',
 'H-300-20119-524353',
 'H-300-20234-781846',
 'H-300-20245-799330',
 'H-300-20260-827308',
 'H-300-20324-919674']

In [10]:
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-19297-107918"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-19310-128739"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-19346-194783"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20119-524353"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20234-781846"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20245-799330"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20260-827308"].CASE_STATUS
h2ajob_certificate[h2ajob_certificate['CASE_NUMBER'] == "H-300-20324-919674"].CASE_STATUS

13068    Determination Issued - Certification
13069        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

12675    Determination Issued - Certification
24180        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

10841    Determination Issued - Certification
24211        Determination Issued - Withdrawn
24213        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

2925     Determination Issued - Certification
24217        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

156                Determination Issued - Certification
23889    Determination Issued - Certification (Expired)
Name: CASE_STATUS, dtype: object

71       Determination Issued - Certification
24165        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

2        Determination Issued - Certification
23959           Determination Issued - Denied
Name: CASE_STATUS, dtype: object

16592    Determination Issued - Certification
24182        Determination Issued - Withdrawn
Name: CASE_STATUS, dtype: object

### Decide to uniformly only keep the first CASE_STATUS because from observing the above cells
#### When a case is certified it is likely and possible for the people to withdraw the application or the certification status becoming expired but it doesnt change the fact that the case is being certified in the beginning (which are the majority shown above)

In [11]:
h2ajob_certificate_dedup = h2ajob_certificate.drop_duplicates(subset='CASE_NUMBER', keep='first')
## check if the depuplication goes well.. (should print empty set)
afterdedup = h2ajob_certificate_dedup.groupby('CASE_NUMBER').CASE_STATUS.nunique() > 1
afterdedup[afterdedup].index.tolist()
## check the case status category again
h2ajob_certificate_dedup.CASE_STATUS.unique()
## delete the withdrawn category because during classification it is arbitrary that withdrawn is fall into
## certified or denied category
h2ajob_certificate_dedup = h2ajob_certificate_dedup[h2ajob_certificate_dedup['CASE_STATUS'] != 'Determination Issued - Withdrawn']
## check the case status category after deleting withdraw category to make sure the deletion is sucessful 
h2ajob_certificate_dedup.CASE_STATUS.unique()

[]

array(['Determination Issued - Certification',
       'Determination Issued - Withdrawn',
       'Determination Issued - Denied',
       'Determination Issued - Partial Certification',
       'Determination Issued - Certification (Expired)',
       'Determination Issued - Partial Certification (Expired)'],
      dtype=object)

array(['Determination Issued - Certification',
       'Determination Issued - Denied',
       'Determination Issued - Partial Certification',
       'Determination Issued - Certification (Expired)',
       'Determination Issued - Partial Certification (Expired)'],
      dtype=object)

### Read in job data 


In [12]:
translated_job_postings=pd.read_csv(DATA_ID_DIR+"translated_job_postings.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Join the job postings of the same case number together 


In [14]:
## join the job postings of the same case number together
concatenate_job=translated_job_postings.groupby(["CASE_NUMBER"])['translatedText'].apply(lambda x: ''.join(x))
# concatenate_job.head()
## change to dataframe
concatenate_job_data= concatenate_job.to_frame().rename(columns={'translatedText':'combined_job_postings'}).reset_index()
concatenate_job_data.head()

,CASE_NUMBER,combined_job_postings
0,H-300-19274-066154,Operating large farm tractors and equipment as...
1,H-300-19274-066174,Employer will furnish free and convenient cook...
2,H-300-19274-066180,INBOUND 1) Advance the worker the required tra...
3,H-300-19274-066199,"For workers who are provided housing, transpor..."
4,H-300-19274-066237,The inbound transportation will be reimbursed ...


In [ ]:
## take a look at some cases to make sure the join works well (before join)
pd.set_option('display.max_colwidth', None)
case1=translated_job_postings[translated_job_postings["CASE_NUMBER"]=="H-300-19274-066154"]
case1["translatedText"]

In [36]:
## take a look at some cases to make sure the join works well (before join)
concatenate_job_data.head(1)
## yeah it seems that everything is joined...

CASE_NUMBER  \
0  H-300-19274-066154   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [37]:
## check if case number is job addendums is unique
concatenate_job_data["CASE_NUMBER"].is_unique      
## true >> unique

True

### Merge translated concatenate_job_data + h2ajob_certificate_dedup

In [38]:
## Finally prepare to merge concatenate_job_data + h2ajob_certificate_dedup
h2ajob_certificate_dedup = h2ajob_certificate_dedup[["CASE_NUMBER","CASE_STATUS"]]
combined_certificate_postings= h2ajob_certificate_dedup.merge(concatenate_job_data, on='CASE_NUMBER', how='inner')
combined_certificate_postings.shape
combined_certificate_postings.head()

(13250, 3)

CASE_NUMBER                           CASE_STATUS  \
0  H-300-20265-835437  Determination Issued - Certification   
1  H-300-20260-827678  Determination Issued - Certification   
2  H-300-20260-827308  Determination Issued - Certification   
3  H-300-20258-821801  Determination Issued - Certification   
4  H-300-20258-821682  Determination Issued - Certification   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Export the merged data


In [39]:
combined_certificate_postings.to_csv(DATA_ID_DIR+'combined_certificate_postings.csv')